# Database Admin 101 - Lab

## Introduction 

In this lab, you'll go through the process of designing and creating a database. From there, you'll begin to populate this table with mock data provided to you.

## Objectives

You will be able to:

* Use knowledge of the structure of databases to create a database and populate it

## The Scenario

You are looking to design a database for a school that will house various information from student grades to contact information, class roster lists and attendance. First, think of how you would design such a database. What tables would you include? What columns would each table have? What would be the primary means to join said tables?

## Creating the Database

Now that you've put a little thought into how you might design your database, it's time to go ahead and create it! Start by import the necessary packages. Then, create a database called **school.sqlite**.

In [1]:
# Import necessary packages
import sqlite3
import pandas as pd


In [2]:
# Create the database school.sqlite 

conn = sqlite3.connect('school.sqlite')

## Create a Table for Contact Information

Create a table called contactInfo to house contact information for both students and staff. Be sure to include columns for first name, last name, role (student/staff), telephone number, street, city, state, and zipcode. Be sure to also create a primary key for the table. 

In [3]:
# Your code here
cur = conn.cursor()
cur.execute('''
CREATE TABLE IF NOT EXISTS contactInfo (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    first_name TEXT,
    last_name TEXT,
    role TEXT,
    telephone TEXT,
    street TEXT,
    city TEXT,
    state TEXT,
    zipcode TEXT
)
''')


## Populate the Table

Below, code is provided for you in order to load a list of dictionaries. Briefly examine the list. Each dictionary in the list will serve as an entry for your contact info table. Once you've briefly investigated the structure of this data, write a for loop to iterate through the list and create an entry in your table for each person's contact info.

In [4]:
# Load the list of dictionaries; just run this cell
import pickle

with open('contact_list.pickle', 'rb') as f:
    contacts = pickle.load(f)

In [5]:
# Iterate over the contact list and populate the contactInfo table here
for contact in contacts:
    cur.execute('''
        INSERT INTO contactInfo
        (first_name, last_name, role, telephone, street, city, state, zipcode)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', (
        contact['firstName'],
        contact['lastName'],
        contact['role'],
        contact['telephone '],
        contact['street'],
        contact['city'],
        contact['state'],
        contact['zipcode ']  
    ))

**Query the Table to Ensure it is populated**

In [6]:
# Your code here 
pd.read_sql("SELECT * FROM contactInfo ", conn)

,id,first_name,last_name,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
6,8,Ed,Lyman,student,5179695576,2910 Simpson Avenue,York,PA,17403
7,17,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
8,18,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
9,19,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303


## Commit Your Changes to the Database

Persist your changes by committing them to the database.

In [7]:
# Your code here
conn.commit()

## Create a Table for Student Grades

Create a new table in the database called "grades". In the table, include the following fields: userId, courseId, grade.

** This problem is a bit more tricky and will require a dual key. (A nuance you have yet to see.)
Here's how to do that:

```SQL
CREATE TABLE table_name(
   column_1 INTEGER NOT NULL,
   column_2 INTEGER NOT NULL,
   ...
   PRIMARY KEY(column_1,column_2,...)
);
```

In [8]:
# Create the grades table
cur.execute('''
CREATE TABLE IF NOT EXISTS grades (
    user_id INTEGER,
    course_id INTEGER,
    grade INTEGER,
    PRIMARY KEY (user_id, course_id)
)
''')


## Remove Duplicate Entries

An analyst just realized that there is a duplicate entry in the contactInfo table! Find and remove it.

In [9]:
# Find the duplicate entry
pd.read_sql('''
SELECT first_name, last_name, COUNT(*) as count
FROM contactInfo
GROUP BY first_name, last_name
HAVING count > 1
''', conn)


,first_name,last_name,count
0,Andrew,Stepp,2
1,Christine,Holden,2
2,Christopher,Warren,2
3,Ed,Lyman,2
4,Jane,Evans,3
5,Linda,Jacobson,2
6,Mary,Raines,2


In [10]:
# Delete the duplicate entry
cur.execute('''
    DELETE FROM contactInfo
    WHERE id NOT IN (
        SELECT MIN(id)
        FROM contactInfo
        GROUP BY first_name, last_name
    )
''')


In [11]:
# Check that the duplicate entry was removed
pd.read_sql("SELECT * FROM contactInfo", conn)


,id,first_name,last_name,role,telephone,street,city,state,zipcode
0,1,Christine,Holden,staff,2035687697,1672 Whitman Court,Stamford,CT,06995
1,2,Christopher,Warren,student,2175150957,1935 University Hill Road,Champaign,IL,61938
2,3,Linda,Jacobson,staff,4049446441,479 Musgrave Street,Atlanta,GA,30303
3,4,Andrew,Stepp,student,7866419252,2981 Lamberts Branch Road,Hialeah,Fl,33012
4,5,Jane,Evans,student,3259909290,1461 Briarhill Lane,Abilene,TX,79602
5,7,Mary,Raines,student,9075772295,3975 Jerry Toth Drive,Ninilchik,AK,99639
6,8,Ed,Lyman,student,5179695576,2910 Simpson Avenue,York,PA,17403


## Updating an Address

Ed Lyman just moved to `2910 Simpson Avenue York, PA 17403`. Update his address accordingly.

In [12]:
# Update Ed's address
cur.execute('''
    UPDATE contactInfo
    SET street = '2910 Simpson Avenue',
        city = 'York',
        state = 'PA',
        zipcode = '17403'
    WHERE first_name = 'Ed' AND last_name = 'Lyman'
''')


In [13]:
# Query the database to ensure the change was made
pd.read_sql("SELECT * FROM contactInfo WHERE first_name='Ed' AND last_name='Lyman'", conn)


,id,first_name,last_name,role,telephone,street,city,state,zipcode
0,8,Ed,Lyman,student,5179695576,2910 Simpson Avenue,York,PA,17403


## Commit Your Changes to the Database

Once again, persist your changes by committing them to the database.

In [14]:
# Your code here
conn.commit()
conn.close()

## Summary

While there's certainly more to do with setting up and managing this database, you got a taste for creating, populating, and maintaining databases! Feel free to continue fleshing out this exercise for more practice. 